# Hand Recognition - data collection
The goal of this tutorial is to use JetBot to make photos of different hand gestures so we can learn the robot to recognise them in next step. Let's begin with importing some necessary libraries:

In [1]:
import traitlets # library enabling some special attributes of Python objects
import ipywidgets.widgets as widgets # library to create widgets in JupyterLab, eg. camera, buttons etc.
from IPython.display import display # library to display widgets in JupyterLab
from jetbot import Camera, bgr8_to_jpeg # library that operates with camera and allows to convert RAW data to .jpg format
import os # library for different system functions

camera = Camera.instance(width=224, height=224) #initiation of 224x224px camera 

image = widgets.Image(format='jpeg', width=224, height=224)  # initiation of the widget with camera (size doesn't have to match)

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg) # transformation of the data from camera to image

display(image) # displaying camera widget

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [2]:
import os

# creation of directories for the pictures of gestures
go_dir = 'dataset_hand_recog/go'
stop_dir = 'dataset_hand_recog/stop'
left_dir = 'dataset_hand_recog/left'
right_dir = 'dataset_hand_recog/right'
circle_dir = 'dataset_hand_recog/circle'
free_dir = 'dataset_hand_recog/free'
blocked_dir = 'dataset_hand_recog/blocked'


# if the directories already exist they won't be created again
try:
    os.makedirs(go_dir)
    os.makedirs(stop_dir)
    os.makedirs(left_dir)
    os.makedirs(right_dir)
    os.makedirs(circle_dir)
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created because they already exist')

Directories not created because they already exist


Now we will create some widgets for buttons that will allow us to grab pictures from the camera

In [3]:
# Let's start with default button layout:
button_layout = widgets.Layout(width='100px', height='100px')
# Now we need to create proper buttons:
# available button_style=
# success: green
# info: blue
# warning: yellow
# danger: blue
# more details in ipywidgets.widgets library
go_button = widgets.Button(description='add go', button_style='success', layout=button_layout)
stop_button = widgets.Button(description='add stop', button_style='danger', layout=button_layout)
left_button = widgets.Button(description='add left', button_style='info', layout=button_layout)
right_button = widgets.Button(description='add right', button_style='info', layout=button_layout)
circle_button = widgets.Button(description='add circle', button_style='warning', layout=button_layout)
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)

# Time for the counters - they will show us how many photos of each category are already in the folders:
go_count = widgets.IntText(layout=button_layout, value=len(os.listdir(go_dir)))
stop_count = widgets.IntText(layout=button_layout, value=len(os.listdir(stop_dir)))
left_count = widgets.IntText(layout=button_layout, value=len(os.listdir(left_dir)))
right_count = widgets.IntText(layout=button_layout, value=len(os.listdir(right_dir)))
circle_count = widgets.IntText(layout=button_layout, value=len(os.listdir(circle_dir)))
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

# Let's display the buttons on the screen:
display(widgets.HBox([go_button, stop_button, left_button, right_button, circle_button, free_button, blocked_button]))
display(widgets.HBox([go_count, stop_count, left_count, right_count, circle_count, free_count, blocked_count]))

Now we will create proper functions to save the frames from the camera to the proper folder:

In [4]:
from uuid import uuid1 # library ___

# function to save the picture in a given folder
def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

# Set of functions to tell the save_snapshot function where to save it and update counter value of given type
def save_go():
    global go_dir, go_count
    save_snapshot(go_dir)
    go_count.value = len(os.listdir(go_dir))
    
def save_stop():
    global stop_dir, stop_count
    save_snapshot(stop_dir)
    stop_count.value = len(os.listdir(stop_dir))
    
def save_left():
    global left_dir, left_count
    save_snapshot(left_dir)
    left_count.value = len(os.listdir(left_dir))
    
def save_right():
    global right_dir, right_count
    save_snapshot(right_dir)
    right_count.value = len(os.listdir(right_dir))
    
def save_circle():
    global circle_dir, circle_count
    save_snapshot(circle_dir)
    circle_count.value = len(os.listdir(circle_dir))

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))

# Let's create "callbacks" - what will happen after pressing the button
# We use "lambda" to ignore some extra parametres from on_click that we don't need
go_button.on_click(lambda x: save_go())
stop_button.on_click(lambda x: save_stop())
left_button.on_click(lambda x: save_left())
right_button.on_click(lambda x: save_right())
circle_button.on_click(lambda x: save_circle())
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

The snippet of code below will display created widgets in one place as it is more convinient to see both camera and buttons in one place.
(In case of modification of the buttons you can copy and paste proper lines from the code above)

In [5]:
display(image)
display(widgets.HBox([go_button, stop_button, left_button, right_button, circle_button, free_button, blocked_button]))
display(widgets.HBox([go_count, stop_count, left_count, right_count, circle_count, free_count, blocked_count]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

After the work is done we need to stop the camera properly to free the resources and make it available for further code.

In [6]:
camera.stop()

If you want to train a model on another machine you can compress created photos and right click .zip file to download it onto your PC. (It can work as a copy of your work as well)

In [1]:
!zip -r -q dataset.zip dataset_hand_recog

Now let's continue in the next notebook - train_model